## read csv

In [1]:
import pandas as pd
import csv
path = 'field_of_study_exercise.csv'

In [2]:
data = pd.read_csv(path)
data.head(10)

,level,level_name,field_of_study,academic_field
0,NaN,NaN,"-, Marketing/Marketing Management, General",marketing
1,NaN,NaN,“Wellness Counseling Certificate”,NaN
2,NaN,NaN,"(BA) Hons Fashion, Fashion/Apparel Design",NaN
3,NaN,NaN,"(BA) Liberal Arts, Liberal Arts and Sciences/L...",NaN
4,NaN,NaN,* Career certificate of Marketing.,NaN
5,NaN,NaN,"1st Class Honour, Information System and Compu...",NaN
6,NaN,NaN,"2:1, Business Management",business management
7,NaN,NaN,"2:1, French and Hispanic Studies",NaN
8,NaN,NaN,"2.1, Graphic Design and Illustration",NaN
9,NaN,NaN,"2.1, Management",NaN


## Data processing

### Remove duplicate rows:

In [3]:
duplicated_record = data.duplicated().sum()
data.drop_duplicates(inplace=True) 
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375 entries, 0 to 374
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   level           322 non-null    object
 1   level_name      312 non-null    object
 2   field_of_study  375 non-null    object
 3   academic_field  141 non-null    object
dtypes: object(4)
memory usage: 14.6+ KB


### Validate Missing Values:

In [4]:
data.info()
null_counts = data.isnull()
print(" -- null count:\n{}".format(null_counts.sum()))
print(" -- null ratios:\n{}".format(null_counts.mean()))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375 entries, 0 to 374
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   level           322 non-null    object
 1   level_name      312 non-null    object
 2   field_of_study  375 non-null    object
 3   academic_field  141 non-null    object
dtypes: object(4)
memory usage: 14.6+ KB
 -- null count:
level              53
level_name         63
field_of_study      0
academic_field    234
dtype: int64
 -- null ratios:
level             0.141333
level_name        0.168000
field_of_study    0.000000
academic_field    0.624000
dtype: float64


When dealing with machine learning and a dataset that is both small and incomplete, filling in missing values (handling NA columns) is a necessary step to maximize the utility of the available data

### Column Cleanup:

In [5]:
# Clean column names
cleaned_data = data.copy()
cleaned_data.columns = cleaned_data.columns.str.strip()
# Display cleaned column names
print(data.columns)

Index(['level', 'level_name', 'field_of_study', 'academic_field'], dtype='object')


### Data Type Conversion:

In [6]:
for column in cleaned_data.columns:
    cleaned_data[column]=cleaned_data[column].astype('str')

### Remove or Replace Unwanted Characters:

In [7]:
import re
# Define a regular expression to remove special characters
pattern = r'[^\w\s]'  # Matches characters that are not alphanumeric or whitespace
# Create an empty dictionary to store column-wise special characters
special_chars = set()
# Iterate through each column in the DataFrame
for col in cleaned_data.columns:
    # Iterate through each element in the column
    for element in data[col].values:
        # Find all special characters using the regular expression
        matches = re.findall(pattern, str(element))
        # Add unique characters to the set
        special_chars.update(matches)

# Print the dictionary showing special characters by column
print("Special characters:")
print(f"{sorted(special_chars)}")

Special characters:
['"', '#', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', ']', '’', '“', '”']


In [8]:

mapping_specific_pattern ={
    '&': 'and',
}

mapping_pattern_based_on_knowledge ={
    'b a': 'ba',
    'b sc': 'bsc',
    'b e':'be',
    'b eng': 'beng',
    'b s e': 'bse',
    'b s': 'bs',
    'bba':'',
    'honours': 'hons',
    'bachelor degree in':'bachelor',
    'bachelor degree':'bachelor',
    'bachelor in':'bachelor',
    'bachelor of':'bachelor',
    'bachelor hons in': 'bachelor hons'
}
# Function to replace substrings regardless of case sensitivity
def replace_substrings(string, replacements):
    pattern = re.compile('|'.join(re.escape(key) for key in replacements.keys()),flags=re.IGNORECASE)
    return pattern.sub(lambda match: replacements[match.group(0).lower()], string)
# Function to clean each entry
def clean_data(field):
    # Split the field by ','
    fields = field.split(',')
    # convert to lower case
    cleaned_fields = [f.lower() for f in fields] 
    # replace special cases:
    cleaned_fields = [replace_substrings(s, mapping_specific_pattern) for s in fields]
    # Clean each split value
    cleaned_fields = [re.sub(r'\W+', ' ', f).strip() for f in cleaned_fields] # Remove punctuation
    # Remove empty fields and lower case
    cleaned_fields = [f.lower() for f in cleaned_fields if f] 
    # Join them back with ','
    cleaned_text =  ','.join(cleaned_fields)
    # Remove multiple spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    return cleaned_text

def mapping_data(field):
    # Split the field by ','
    fields = field.split(',')
    # mapping specific patterns:
    cleaned_fields = [replace_substrings(s, mapping_pattern_based_on_knowledge) for s in fields]
    # Join them back with ','
    cleaned_text =  ','.join(cleaned_fields)
    return cleaned_text

cleaned_data['field_of_study'] = cleaned_data['field_of_study'].apply(clean_data)
before_mapping = cleaned_data.copy()
cleaned_data['field_of_study'] = cleaned_data['field_of_study'].apply(mapping_data)
cleaned_data['academic_field'] = cleaned_data['academic_field'].apply(clean_data)
cleaned_data['level'] = cleaned_data['level'].apply(clean_data)
cleaned_data['level_name'] = cleaned_data['level_name'].apply(clean_data)
cleaned_data.replace("nan", pd.NA, inplace=True)

In [9]:
complete_data = cleaned_data.dropna()
complete_data.to_csv('complete_data.csv',index=False,quoting=csv.QUOTE_ALL)

### Validate and Standardize Values:

#### K-Nearest Neighbors (KNN) imputation

In [10]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

knn_data=cleaned_data.copy()
# Introducing a missing value in 'academic_field' column
knn_data.loc[2, 'academic_field'] = None
# Encode the 'academic_field' column
label_encoder = LabelEncoder()
knn_data['academic_field_encoded'] = label_encoder.fit_transform(knn_data['academic_field'].astype(str))
knn_data['academic_field_encoded_feature_exatraction'] = label_encoder.fit_transform(knn_data['academic_field'].astype(str))

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(','))
tfidf_matrix = tfidf_vectorizer.fit_transform(knn_data['field_of_study'])

# Prepare feature matrix with TF-IDF features
feature_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
feature_matrix['academic_field_encoded_feature_exatraction'] = knn_data['academic_field_encoded_feature_exatraction']

# Prepare the feature matrix
data_encoded = pd.get_dummies(knn_data['field_of_study'])
data_encoded['academic_field_encoded'] = knn_data['academic_field_encoded']


data_encoded = pd.concat([data_encoded, feature_matrix], axis=1)
# Impute missing values using KNNImputer
imputer = KNNImputer(n_neighbors=20) 
data_imputed = imputer.fit_transform(data_encoded)

# Decode the 'academic_field' column back to original labels
knn_data['academic_field_imputed'] = label_encoder.inverse_transform(data_imputed[:, -1].round().astype(int))

knn_data.drop(['academic_field_encoded','academic_field_encoded_feature_exatraction'], axis=1, inplace=True)
knn_data.to_csv('knn_data.csv',index=False,quoting=csv.QUOTE_ALL)

As a result, the data from the CSV file still does not contain enough records for some basic ML models to handle the missing data. Manual filling may be necessary.

# SUMMARY:

In [11]:
print("""
## Introduction
This report summarizes the data cleaning process for the academic category. 
The primary goal was to ensure data quality by removing inconsistencies, handling missing values, and standardizing formats.
      """)

print("## Data Overview\n")
# Overview of the dataset
initial_record_count = data.shape[0]
initial_column_count = data.shape[1]

print(f"Initial number of records: {initial_record_count}")
print(f"Initial number of columns: {initial_column_count}")

print("\n\n## Data Cleaning Steps\n")
null_counts = data.isnull()
print(" -- null count:\n{}".format(null_counts.sum()))
print(" -- null ratios:\n{}".format(null_counts.mean()))
print(f"duplicated records: {duplicated_record}")

print("\n### Handling Missing Values:\n")
# Initial count of missing values
missing_values_initial = data.isnull().sum().sum()
records_removed_missing_values = initial_record_count - complete_data.shape[0]
print(f"Records removed due to missing values: {records_removed_missing_values}")
print(f"Remaining Records: {complete_data.shape[0]}")

print("\n### Standardization\n")

#lower case the name of columns:
original_column = [col for col in data.columns]
lowercase_column = [col for col in complete_data.columns]
print("Convert all column names to lower case:")
print(f"original_columns: {original_column}")
print(f"output columns: {lowercase_column}")

total_occurrences = []
mapped_occurrences = []

print("\n### Data Mapping\n")
for key in mapping_pattern_based_on_knowledge.keys():
    count = sum(before_mapping['field_of_study'].str.count(re.escape(key), flags=re.IGNORECASE))
    # print(f"number of '{key}' occurrences:{total_occurrences}  ")
    mapped_occurrences.append({key:count})
for val in mapping_pattern_based_on_knowledge.values():
        count = sum(cleaned_data['field_of_study'].str.count(re.escape(val), flags=re.IGNORECASE))
        # print(f"number of '{val}' occurrences:{total_occurrences}  ")
        total_occurrences.append({val:count})

for index, (key, value) in enumerate(mapping_pattern_based_on_knowledge.items()):
    print(f"mapping ratio '{key}' to '{value}': {round(mapped_occurrences[index].get(key)/total_occurrences[index].get(value),3)}")


print("\n\n## Cleaned Data Overview:\n")

print(f"Number of rows: {complete_data.shape[0]} \nColumns: {complete_data.shape[1]}")
print("The first few rows:")
complete_data.head(10)


## Introduction
This report summarizes the data cleaning process for the academic category. 
The primary goal was to ensure data quality by removing inconsistencies, handling missing values, and standardizing formats.
      
## Data Overview

Initial number of records: 375
Initial number of columns: 4


## Data Cleaning Steps

 -- null count:
level              53
level_name         63
field_of_study      0
academic_field    234
dtype: int64
 -- null ratios:
level             0.141333
level_name        0.168000
field_of_study    0.000000
academic_field    0.624000
dtype: float64
duplicated records: 0

### Handling Missing Values:

Records removed due to missing values: 252
Remaining Records: 123

### Standardization

Convert all column names to lower case:
original_columns: ['level', 'level_name', 'field_of_study', 'academic_field']
output columns: ['level', 'level_name', 'field_of_study', 'academic_field']

### Data Mapping

mapping ratio 'b a' to 'ba': 0.172
mapping ratio 'b sc' to 

,level,level_name,field_of_study,academic_field
64,bachelor,bachelor,"b communications public relations,marketing",marketing
65,bachelor,bachelor,"bsc,chemistry general",chemistry
69,bachelor,bachelor,"beng,electrical engineering",electrical engineering
70,bachelor,bachelor,"bsc,computer science networking techniques and...",computer science
72,bachelor,bachelor,"ba 2nd upper class hons,history",history
75,bachelor,bachelor,"ba hons 2nd upper,english literature",english literature
79,bachelor,bachelor,"ba,computer science",computer science
87,bachelor,bachelor,"be,computer science",computer science
89,bachelor,bachelor,"be cs,computer science",computer science
93,bachelor,bachelor,"beng,electrical engineering",electrical engineering


In [12]:
print("data type of each columns:")
complete_data.dtypes

data type of each columns:


level             object
level_name        object
field_of_study    object
academic_field    object
dtype: object

In [13]:
print("descriptive statistics of numeric columns:")
complete_data.describe()

descriptive statistics of numeric columns:


,level,level_name,field_of_study,academic_field
count,123,123,123,123
unique,1,2,115,29
top,bachelor,bachelor,"bachelor arts ba,sociology",business management
freq,123,120,2,17
